In [222]:
import pandas as pd
from utils import data_util as du, preproc_utils as pu, extractor_util as exu, estimator_util as esu
import importlib
from typing import List, Dict, Any
import wrappers as wrp
import os

In [4]:
DATA_SRC_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data'
EXT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external'
INT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/internal'

In [213]:
# Usage example
dataset = du.DatasetLoader(EXT_DATA_DIR)
train_data = dataset.load_all_scenes()


In [199]:
# Usage
config = pu.PreprocessConfig(size=(512, 512), normalize=False)
preprocessor = pu.ImagePreprocessor(config)

In [217]:
default_sift_config = exu.SIFTConfig()
extractor_config = exu.FeatureExtractorConfig(default_sift_config)
extractor = exu.FeatureExtractor(config=extractor_config)

In [244]:
dataset = wrp.load_dataset(EXT_DATA_DIR)

In [247]:
dataset = wrp.preprocess_data(dataset, preprocessor, extractor)

In [256]:
dataset = wrp.extract_features(dataset, extractor)

In [ ]:
wrp.match_features(dataset, extractor)

In [265]:
scene_data = dataset.scenes_data['brandenburg_gate']
valid_pairs = scene_data.covisibility[scene_data.covisibility['covisibility'] > 0.1]

In [ ]:
for row in valid_pairs.iterrows():
    print(row)

In [124]:
bf_config = esu.BFMatcherConfig(knn=True)
matcher_config = esu.MatcherConfig(bf_config)

In [125]:
matcher = esu.FeatureMatcher(matcher_config)

In [291]:
importlib.reload(wrp)
importlib.reload(du)
importlib.reload(exu)
importlib.reload(esu)

<module 'utils.estimator_util' from '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/utils/estimator_util.py'>

In [283]:
valid_pairs = scene_data.covisibility[scene_data.covisibility['covisibility'] > 0.1]

In [292]:
wrp.match_features(dataset, matcher)

IndexError: tuple index out of range

In [126]:
matches = matcher.match_features(i1f, i2f)

In [127]:
valid, kp1, kp2 = matcher.filter_lowe_matches(matches, i1f, i2f)

In [141]:
type(kp1[0])



cv2.KeyPoint

In [143]:
es_alg_conf = esu.RANSACConfig()
es_conf = esu.EstimatorConfig('RANSAC', es_alg_conf)


In [144]:
fe = esu.FundamentalMatrixEstimator(kp1, kp2, es_conf)

In [145]:
fe.estimate()

array([[ 2.19455369e-06, -1.28450711e-04,  5.75317978e-02],
       [ 1.10165394e-04,  6.54828648e-07, -2.44261228e-02],
       [-6.91245930e-02,  3.13106405e-02,  1.00000000e+00]])

In [149]:
inl = fe.get_inliers()

In [ ]:
# Convert inliers to a set of tuples for easy comparison
inliers_set = set(map(tuple, inl[0]))
print(f"Found {len(inliers_set)} inliers from {len(inl[0])} inliers")

# Find outliers by checking which keypoints in kp1 are not in inliers
outliers = [kp for kp in kp1 if (kp.pt[0], kp.pt[1]) not in inliers_set]

print(f"Inliers count: {len(inliers_set)} out of {len(kp1)}")
print("Total outliers found: ")
print(len(set(outliers)))

Found 24 inliers from 28 INLIERS
Inliers count: 24 out of 59
Total outliers found: 
31
